### Prompt

The question you are trying to answer is: In what future month will a new breed of rabbits to exceed the human population 2:1? There are a few moving pieces:
1) We will start with 1 female and 1 male rabbit born today (0 months old)
2) Each month, each female produces a litter of 4 bunnies: 1 male and 3 females
3) A female cannot start to reproduce until she has matured to 4 months of age
4) All rabbits live only to 24 months in age. (Alive months 0-23, die on the start of the 24th month)

Don't forget human population grows too! Assume a 0.058% month over month human population growth!

In [1]:
# importing lib

import pandas as pd

In [49]:
# creating data

data_dict = {'male_pop' : '1',
            'female_pop' : '1',
            'age' : '0',
            'date' : '2018-07-02',
            'human_pop' : '326766748'}

df = pd.DataFrame([data_dict])

df

,male_pop,female_pop,age,date,human_pop
0,1,1,0,2018-07-02,326766748


In [50]:
df.dtypes

male_pop      object
female_pop    object
age           object
date          object
human_pop     object
dtype: object

In [51]:
# converting data to datetime

df['date'] = pd.to_datetime(df['date'])

In [52]:
# converting numeric fields to numeric

df['male_pop'] = df['male_pop'].astype('int64')
df['female_pop'] = df['female_pop'].astype('int64')
df['age'] = df['age'].astype('int64')
df['human_pop'] = df['human_pop'].astype('int64')

In [53]:
df.dtypes

male_pop               int64
female_pop             int64
age                    int64
date          datetime64[ns]
human_pop              int64
dtype: object

In [74]:
# creating iterative function

def rabbits(df):
    
    human_pop = df['human_pop'].max()
    rabbit_pop = (df['male_pop'] + df['female_pop']).max()
    
    while human_pop * 2 >= rabbit_pop:
    
        df_survived = df[df['age'] < 24] # filtering out dead rabbits

        df_survived['age'] += 1 # adding 1 to age

        df_survived['date'] += pd.DateOffset(months=1) # adding 1 month to date

        df_survived['human_pop'] = df_survived['human_pop'] * 1.00058 # incrementing human population
    
        # filtering rabbits above 4 
        
        df_bearing = df_survived[df_survived['age'] > 4]
        
        # bearing rabbits
        
        df_bearing_new = df_bearing.groupby(['date', 'human_pop']).agg(age = ('age', 'max'), male_pop = ('male_pop', 'sum'),
                                                      female_pop = ('female_pop', 'sum')).reset_index()
        
        df_bearing_new['age'] = 0
        df_bearing_new['male_pop'] = df_bearing_new['female_pop']
        df_bearing_new['female_pop'] = df_bearing_new['female_pop'] * 3
        
        # filtering rabbits below 4
        
        df_unbearing = df_survived[df_survived['age'] <= 4]
        
        # unioning bearing, new, and unbearing
        
        union_df = [df_bearing, df_bearing_new, df_unbearing]
        
        df_unioned = pd.concat(union_df)
        
        # transforming data to include total rabbit pop in each row
        
        df_unioned['rabbit_pop'] = df_unioned['female_pop'] + df_unioned['male_pop']
        
        # resetting variables
        
        rabbit_pop = df_unioned['rabbit_pop'].sum()
        
        human_pop = df_unioned['human_pop'].max()
        
        df = df_unioned
        
    df = df.groupby(['human_pop', 'date']).agg(rabbit_pop = ('rabbit_pop', 'sum'))
    
    return df

In [75]:
rabbits(df)

C:\Users\CharlesYi\AppData\Local\Temp\ipykernel_4484\2266716140.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_survived['age'] += 1 # adding 1 to age
C:\Users\CharlesYi\AppData\Local\Temp\ipykernel_4484\2266716140.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_survived['date'] += pd.DateOffset(months=1) # adding 1 month to date
C:\Users\CharlesYi\AppData\Local\Temp\ipykernel_4484\2266716140.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

C:\Users\CharlesYi\AppData\Local\Temp\ipykernel_4484\2266716140.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_survived['age'] += 1 # adding 1 to age
C:\Users\CharlesYi\AppData\Local\Temp\ipykernel_4484\2266716140.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_survived['date'] += pd.DateOffset(months=1) # adding 1 month to date
C:\Users\CharlesYi\AppData\Local\Temp\ipykernel_4484\2266716140.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

,,rabbit_pop
human_pop,date,
3.361839e+08,2022-08-02,785219160
